# REST API Search Endpoint

Método de consulta de tweets históricos (hasta una semana) basándose en varios criterios, es muy adecuado para un análisis estático.

## Carga inicial de datos

In [43]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests_oauthlib import OAuth1
consumer_key='ynSB0dFvqPl3xRU7AmYk39rGT'
consumer_secret='6alIXTKSxf0RE57QK3fDQ8dxdvlsVr1IRsHDZmoSlMx96YKBFD'
access_token='966591013182722049-BVXW14Hf5s6O2oIwS3vtJ3S3dOsKLbY'
access_token_secret='829DTKPjmwsSytmp1ky9fMCJkjV0LZ04TbL9oqHGV6cDm'

Parte en la que se importa las librerías necesarias.

In [44]:
q = 'premier league -filter:retweets AND -filter:replies'
url = 'https://api.Twitter.com/1.1/search/tweets.json'
pms = {'q' : q, 'count' : 100, 'lang' : 'en', 'result_type': 'recent'} 
auth = OAuth1(consumer_key, consumer_secret, access_token,access_token_secret)
#res = requests.get(url, params = pms, auth=auth)
#esta parte del código se utiliza realmente en la paginación

Sección de código que realiza la consulta **q** mediante la api de twitter utilizando los parámetros **pms**. Después, se crea una variable **auth** que corresponde a los datos exigidos por la API para conectarse que son *Las claves y los tokens de acceso.*
Por último se realiza la petición utilizando todos los datos necesarios por medio de una función de la librería requests **(request.get)** el resultado se vuelca en la variable **res**.

In [45]:
#tweets = res.json()
#esta parte del código se utiliza realmente en la paginación

Si la conección es correcta se recuperarán varios documentos que se pueden convertir a json. En el formato json es en el cual extraeremos la información que nos importa.

## Carga de datos para más de 100 tweets (Paginación)

Como twitter te permite obtener hasta 100 tweets por llamada, si queremos recuperar más, necesitamos recordar los IDs de los tweets ya descargados para no repetir tweets en las siguientes llamadas. Esto es la paginación.

In [46]:
database_name = "baseDeDatos"
collection_name = "coleccion"
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client[database_name]
collection = db[collection_name]

**MongoDB** *es una base de datos NoSQL ágil orientado a documentos, esto significa que en lugar de guardar los datos en tablas como se hace en las bbdd relacionales, MongoDB guarda estructuras de datos en documentos similares a JSON con un esquema dinámico.*
En nuestro caso importamos la librería para usar el cliente y creamos una base de datos y una colección a raíz de la base de datos. Ya cambiaré los nombres de **database_name** y **collection_name**. <font color='red'>falta establecer el nombre de la base de datos y el nombre de la colección</font>

In [47]:
pages_counter = 0
number_of_pages = 100
while pages_counter < number_of_pages:
    pages_counter += 1
    res = requests.get(url, params = pms, auth=auth)
    print("Connection status: %s" % res.reason)
    tweets = res.json()
    ids = [i['id'] for i in tweets['statuses']]
    pms['max_id'] = min(ids) - 1
    collection.insert_many(tweets['statuses'])

Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection status: OK
Connection

KeyError: 'statuses'

El código para la paginación consiste en un bucle que se repetirá tantas veces como el número aproximado de tweets que queramos recuperar de twitter. Dentro del bucle se realiza la *consulta* (**requests.get**) y se guarda en la variable tweets con formato json (igual que se hacía antes de la paginación)
Después, para implementar lo que es realmente la paginación; 
- Se recolectan todos los **ids** de los tweets recogidos mediante el **get**.
- Se almacena en **i** todos los **ids** de todos los tweets.
- Se establece en **pms** como el *id mayor*, el *id mínimo* de **i** -1 (porque se incluiría y después se duplica)
- Por último se inserta en la base de datos (Con **insert_many**) todos los tweets que han sido recuperados en esta *ronda*

In [48]:
#client = MongoClient('mongodb://localhost:27017/')
#db = client['db']
#collection = db['collection']
documents = []
for doc in collection.find():
    documents.append(doc)

Lo que se hace a continuación es utilizar la base de datos de mongo en la que se ha estado almacenando los tweets para volcar la información de forma tabular, donde las columnas indican los nombres de los nodos del documento y las filas representan los datos.


In [49]:
df = pd.DataFrame(documents)
df

,_id,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,text,truncated,user,withheld_in_countries
0,5ae84bdeb72f4e2b14e77cc3,None,None,Tue May 01 11:13:16 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 991273896065294336, 'id_str'...",0,False,None,991274350421692416,...,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Gloucestershire Hellenic League Premier Divisi...,False,"{'id': 164414121, 'id_str': '164414121', 'name...",NaN
1,5ae84bdeb72f4e2b14e77cc4,None,None,Tue May 01 11:13:08 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,991274317630656513,...,NaN,NaN,NaN,0,False,"<a href=""http://www.facebook.com/twitter"" rel=...",Cristante A Ready-Made Star For Arsenal! Find ...,False,"{'id': 1724406476, 'id_str': '1724406476', 'na...",NaN
2,5ae84bdeb72f4e2b14e77cc5,None,None,Tue May 01 11:13:06 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 991274306675060737, 'id_str'...",0,False,None,991274308264693762,...,NaN,NaN,NaN,0,False,"<a href=""http://www.crowdfireapp.com"" rel=""nof...",Prem: Top goals of Week 36 - ESPN FC\nhttps://...,False,"{'id': 1663956560, 'id_str': '1663956560', 'na...",NaN
3,5ae84bdeb72f4e2b14e77cc6,None,None,Tue May 01 11:11:32 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,991273910997078017,...,NaN,NaN,NaN,0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...","Chelsea target, Sarri hints at dumping Napoli ...",False,"{'id': 3334046350, 'id_str': '3334046350', 'na...",NaN
4,5ae84bdeb72f4e2b14e77cc7,None,None,Tue May 01 11:10:58 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 991273767023271936, 'id_str'...",0,False,None,991273769070084096,...,NaN,NaN,NaN,0,False,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",Bosnian Source Claims Liverpool Assistant Zelj...,False,"{'id': 841031560673411073, 'id_str': '84103156...",NaN
5,5ae84bdeb72f4e2b14e77cc8,None,None,Tue May 01 11:10:37 +0000 2018,"{'hashtags': [{'text': 'stokecity', 'indices':...",NaN,0,False,None,991273682705289217,...,NaN,NaN,NaN,0,False,"<a href=""http://3due.uno"" rel=""nofollow"">3due....",There will be a reckoning at Stoke City once t...,True,"{'id': 4097611282, 'id_str': '4097611282', 'na...",NaN
6,5ae84bdeb72f4e2b14e77cc9,None,None,Tue May 01 11:10:32 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,991273662547398656,...,NaN,NaN,NaN,0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tottenham Hotspur can still finish second in P...,False,"{'id': 2984771926, 'id_str': '2984771926', 'na...",NaN
7,5ae84bdeb72f4e2b14e77cca,None,None,Tue May 01 11:10:29 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,None,991273650627207168,...,NaN,NaN,NaN,1,False,"<a href=""https://about.twitter.com/products/tw...",Until the weekend Zeljko Buvac was mainly know...,True,"{'id': 19292816, 'id_str': '19292816', 'name':...",NaN
8,5ae84bdeb72f4e2b14e77ccb,None,None,Tue May 01 11:10:25 +0000 2018,"{'hashtags': [{'text': 'PremierLeague', 'indic...",NaN,0,False,None,991273631446716418,...,NaN,NaN,NaN,0,False,"<a href=""http://www.football-addict.com/en/new...",#PremierLeague Team of the Season: Gary Nevill...,False,"{'id': 824282039939727361, 'id_str': '82428203...",NaN
9,5ae84bdeb72f4e2b14e77ccc,None,None,Tue May 01 11:10:09 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,2,False,None,991273565877129216,...,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",So Jimmy Nicholl takes charge of Rangers until...,True,"{'id': 296846015, 'id_str': '296846015', 'name...",NaN


En este paso se crea un Dataframe usando la lista de documentos que hemos construido.

# Limpieza de datos

Nos gustaría analizar los sentimientos de los tweets que son escritos por la gente desde sus diferentes dispositivos. Por lo tanto, debemos deshacernos de los tweets compuestos por bots, páginas web, servicios de envío automatizado, etc. No podemos identificar esos tweets al 100%, pero es bastante buena suposición sería seleccionar tweets publicados desde dispositivos físicos, es decir, iPhones, Teléfonos Android, ordenadores de sobremesa y portátiles. 
Los documentos de Twitter tienen una característica interesante, ya que mantienen la información sobre la fuente de la creación del tweet. Cada vez que alguien usa un dispositivo para componer un tweet, el se mantiene la información al respecto.

In [50]:
df['tweet_source'] = df['source'].apply(lambda x:
                                        BeautifulSoup(x).get_text())

C:\Anaconda\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Con esta linea de código vamos a crear una nueva columna en el data Frame df que se llama tweet_source y ahí vamos a poner lo que había en la columna source pero con la información que necesitamos, es decir, la fuente del tweet.
Ahora bien, sólo nos interesan los dispositivos. La información sobre los dispositivos es muy clara: encontraremos fuentes de tweets como Twitter para iPhone, Twitter para Android, Twitter Web Client, Twitter para BlackBerry, Twitter para Mac, Twitter para Windows, etc. Los nombres de los dispositivos comienzan con la palabra "Twitter". Utilizaremos esta propiedad para abordar nuestra segunda cuestión.

In [52]:
devices = list(set(df[df['tweet_source'].str.startswith('Twitter')]['tweet_source']))

De esta manera se extrae en la variable devices todas las fuentes de los tweets que comiencen por Twitter, es decir, todas las que vienen de dispositivos.